In [1]:
import pandas as pd
import numpy as np

In [2]:
car_df = pd.read_csv("./archive/v2.csv")
car_df.shape

/var/folders/wj/kg65sk251rz1g4h8_694mm9r0000gn/T/ipykernel_27281/4169038337.py:1: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  car_df = pd.read_csv("./archive/v2.csv")


(3120272, 17)

In [3]:
car_df.head()

,tripID,deviceID,timeStamp,accData,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos
0,1,0.0,2017-12-22 18:43:05,10c0f8e00448fa18c80515d30000000000000000000000...,24.2612,0.0,66.0,0.0,28.6275,40.0,97.0,0.0,0.0,1010.75,23.0,0.0,0.0
1,1,0.0,2017-12-22 18:43:06,1138f8c804780a1ebdf718bcf919d10617c8e301b31017...,23.15,0.0,66.0,0.0,33.7255,40.0,98.0,0.0,0.0,815.5,21.0,0.0,0.0
2,1,0.0,2017-12-22 18:43:07,10f0f89804480612c30010c30714ce0520b7f41dbdf118...,18.7052,0.0,66.0,0.0,43.1373,40.0,98.0,0.0,0.0,862.25,17.0,0.0,0.0
3,1,0.0,2017-12-22 18:43:08,10d0f84804480d15bd0210c9f822c80017caf81ccd0517...,16.4828,0.0,66.0,0.0,41.5686,40.0,97.0,0.0,0.0,817.0,17.0,0.0,0.0
4,1,0.0,2017-12-22 18:43:09,1090f8c80480041dc9081cc50815c60511c60112c40514...,17.4088,0.0,66.0,0.0,43.1373,40.0,97.0,0.0,0.0,804.25,15.0,0.0,0.0


### Rename raw columns to improve readability

In [4]:
column_rename_dict = {
    'tripID': 'trip_id',
    'deviceID': 'device_id',
    'timeStamp': 'timestamp',
    'accData': 'accelerometer',
    'battery': 'battery_volt',
    'cTemp': 'coolant_temp',
    'dtc': 'diagnostic_trouble_code',
    'eLoad': 'engine_load',
    'iat': 'intake_air_temp',
    'maf': 'air_mass_rate',
    'rpm': 'revolutions_per_min',
    'tAdv': 'timing_advance',
    'tPos': 'throttle_position',
}

car_df = car_df.rename(columns=column_rename_dict)

### Data Dictionary

**Basic Information**

| Column | Description |
| :--- | :----------- |
| device_id | A device id is a car. One Device can have multiple trip ids. device_id + trip_id are unique trips.|
| trip_id | One id is 1 trip. A trip begins when car engine is switched on and ends when the engine is switched off. |
| timestamp | Data collection time. |
| battery_volt | The battery voltage corresponds to voltage of the battery installed in Car, which supplies electrical energy to a motor vehicle. |
| diagnostic_trouble_code | Number of diagnostic trouble codes. DTC's, or Diagnostic Trouble Codes, are used by automobile manufacturers to diagnose problems related to the vehicle. |
| accelerometer | Accelerometer and Magnetometer sensor data. The data is collected from the OBD device. Values are in terms of G-force. The data is across X, Y, Z axis where X-axis is horizontal, Y-axis is vertical, and Z-axis is the direction of movement of the car. |


**Speed related sensors**

| Column | Description |
| :--- | :----------- |
| gps_speed | The speed in kmph (kilometers per hour) as noted from GPS sensor. |
| speed | Speed data as collected from OBD device mounted in the car. |
| kpl | KMPL is mileage in kilometres per litre. It is a derived metric derived from speed and fuel to air mass flow ratio. This ratio is constant in case of Petrol cars while changes for other Fuel types. Hence, the KMPL value is accurate for petrol cars, and contain some error in case of other fuel types. |
| revolutions_per_min | engine RPM. The number of turns in one minute. |


**Temperature related sensors**

| Column | Description |
| :--- | :----------- |
| coolant_temp | The Temperature of the engine coolant of an internal combustion engine. The normal operating temperature for most engines is in a range of 90 to 104 degree Celsius (195 to 220 degrees Fahrenheit). |
| intake_air_temp | The Intake Air Temperature sensor (IAT) has been utilised as an Engine Control Unit (ECU) input signal, as a requirement for calculating the Air Mass volume for the incoming air charge. This is, to assist in determining the correct engine fuel requirement to suit the operating air temperature. |
| timing_advance | Timing advance refers to the number of degrees before top dead center (BTDC) that the spark will ignite the air-fuel mixture in the combustion chamber during the compression stroke. |


**Air mass flow related sensors**

| Column | Description |
| :--- | :----------- |
| imap | The manifold absolute pressure sensor (MAP sensor) is one of the sensors used in an internal combustion engine's electronic control system. The MAP sensor sensor provides instantaneous manifold pressure information to the engine's electronic control unit (ECU). The data is used to calculate air density and determine the engine's air mass flow rate, which in turn determines the required fuel metering for optimum combustion (see stoichiometry) and influence the advance or retard of ignition timing. |
| engine_load | Engine load measures how much air (and fuel) you're sucking into the engine and then compares that value to the theoretical maximum. |
| throttle_position | A throttle position sensor (TPS) is a sensor used to monitor the air intake of an engine. |
| air_mass_rate | A mass (air) flow sensor (MAF) is used to find out the mass flow rate of air entering a fuel-injected internal combustion engine. The air mass information is necessary for the engine control unit (ECU) to balance and deliver the correct fuel mass to the engine. |

### Noisy points

In [5]:
noise_index = car_df["device_id"] == "deviceID"
car_df[noise_index].head()

,trip_id,device_id,timestamp,accelerometer,gps_speed,battery_volt,coolant_temp,diagnostic_trouble_code,engine_load,intake_air_temp,imap,kpl,air_mass_rate,revolutions_per_min,speed,timing_advance,throttle_position
106461,tripID,deviceID,timeStamp,accData,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos
376682,tripID,deviceID,timeStamp,accData,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos
630522,tripID,deviceID,timeStamp,accData,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos
631504,tripID,deviceID,timeStamp,accData,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos
676250,tripID,deviceID,timeStamp,accData,gps_speed,battery,cTemp,dtc,eLoad,iat,imap,kpl,maf,rpm,speed,tAdv,tPos


In [6]:
# Remove noise
car_df = car_df[~noise_index]
print("Data Size after removing noise: {}".format(car_df.shape))

Data Size after removing noise: (3120240, 17)


### Data type casting

In [7]:
first_datatype_casting = {
    'device_id': np.float32,
    'trip_id': np.float32,
    'timestamp': "datetime64[ns]",
    'battery_volt': np.float32,
    'diagnostic_trouble_code': np.float32,
    'accelerometer': str,
    'gps_speed': np.float32,
    'speed': np.float32,
    'kpl': np.float32,
    'revolutions_per_min': np.float32,
    'coolant_temp': np.float32,
    'intake_air_temp': np.float32,
    'timing_advance': np.float32,
    'imap': np.float32,
    'engine_load': np.float32,
    'throttle_position': np.float32,
    'air_mass_rate': np.float32,
}

# Cast integers for memory efficiency
second_datatype_casting = {
    'device_id': np.int8,
    'trip_id': np.int8,
    'diagnostic_trouble_code': np.int8,
    'speed': np.int8,
    'coolant_temp': np.int8,
    'intake_air_temp': np.int8,
    'imap': np.int8,
}

car_df = car_df.astype(first_datatype_casting).astype(second_datatype_casting)
car_df.dtypes

trip_id                              int8
device_id                            int8
timestamp                  datetime64[ns]
accelerometer                      object
gps_speed                         float32
battery_volt                      float32
coolant_temp                         int8
diagnostic_trouble_code              int8
engine_load                       float32
intake_air_temp                      int8
imap                                 int8
kpl                               float32
air_mass_rate                     float32
revolutions_per_min               float32
speed                                int8
timing_advance                    float32
throttle_position                 float32
dtype: object

### Timestamp Range and sort signal history by timestamp

In [8]:
car_df['timestamp'].min(), car_df['timestamp'].max()

(Timestamp('2017-11-18 16:23:30'), Timestamp('2018-01-31 23:18:50'))

In [9]:
car_df = car_df.sort_values(['device_id', 'trip_id', 'timestamp'], ascending=True)

### Duplicates
Duplicate signals might be useful.

In [10]:
print("Duplicate Rate: {:,.3f}%".format(
    np.sum(car_df.duplicated()) / car_df.shape[0] * 100))

# Drop all duplicates
car_df = car_df.drop_duplicates()
print("Data Size after removing duplicates: {}".format(car_df.shape))

Duplicate Rate: 26.543%
Data Size after removing duplicates: (2292031, 17)


### Missing Values

In [11]:
car_df.isnull().values.any()

False

### Add day driving / night drive labels 

In [12]:
car_df['timestamp'] = pd.to_datetime(car_df['timestamp'])

In [13]:
night_driving_index = (car_df['timestamp'].dt.hour >= 23) | (car_df['timestamp'].dt.hour <= 4)
day_driving_index = ~night_driving_index

print("night driving pct: {:,.3f}%\nday driving pct: {:,.3f}%".format(
    sum(night_driving_index) / car_df.shape[0] * 100,
    sum(day_driving_index) / car_df.shape[0] * 100))

car_df.loc[night_driving_index, 'night_driving'] = 1
car_df.loc[day_driving_index, 'night_driving'] = 0

night driving pct: 0.782%
day driving pct: 99.218%


In [14]:
car_df['device_id'].value_counts()

12    518906
10    484612
9     312916
5     267328
0     184242
7     172654
3     147706
16     83043
8      43185
2      27155
6      21224
1      10272
4       9878
11      7929
14       981
Name: device_id, dtype: int64

### Check signal frequency for each sensor
In over 99% of the time, each sensor had only 1 unique signal. It can be considered as 1 Hz.

This means 1 timestamp can be considered as 1 time step.
It requires signal preprocessing to only keep the last signal for each timestamp.

In [15]:
def check_signal_frequency(df):
    # Calculate the signal frequency of each sensor (number of unique values per timestamp)
    for c in ['accelerometer', 'gps_speed',
           'battery_volt', 'coolant_temp', 'diagnostic_trouble_code',
           'engine_load', 'intake_air_temp', 'imap', 'kpl', 'air_mass_rate',
           'revolutions_per_min', 'speed', 'timing_advance', 'throttle_position',]:

        signal_frequency = car_df[['device_id', 'trip_id', 'timestamp', c]]\
            .drop_duplicates()\
            .groupby(['device_id', 'trip_id', 'timestamp'])\
            .count()

        print("{} in {:,.3f}% time had 1 signal per second (1 Hz)".format(
            c, 
            signal_frequency[signal_frequency[c] == 1].shape[0] / signal_frequency.shape[0] * 100))

In [16]:
check_signal_frequency(car_df)

accelerometer in 99.226% time had 1 signal per second (1 Hz)
gps_speed in 100.000% time had 1 signal per second (1 Hz)
battery_volt in 99.934% time had 1 signal per second (1 Hz)
coolant_temp in 99.960% time had 1 signal per second (1 Hz)
diagnostic_trouble_code in 100.000% time had 1 signal per second (1 Hz)
engine_load in 99.651% time had 1 signal per second (1 Hz)
intake_air_temp in 99.972% time had 1 signal per second (1 Hz)
imap in 99.757% time had 1 signal per second (1 Hz)
kpl in 99.697% time had 1 signal per second (1 Hz)
air_mass_rate in 99.691% time had 1 signal per second (1 Hz)
revolutions_per_min in 99.625% time had 1 signal per second (1 Hz)
speed in 99.745% time had 1 signal per second (1 Hz)
timing_advance in 99.934% time had 1 signal per second (1 Hz)
throttle_position in 99.879% time had 1 signal per second (1 Hz)


### Keep the last signal for each timestamp
Some timestamps had multiple different signals. Since all sensors are considered as 1 Hz, **it's good to only keep the last signal and remove the rest.**

In [17]:
redundant_signals = car_df.duplicated(subset=['device_id', 'trip_id', 'timestamp'], 
                                      keep='last')

print("{:,.3f}% redundant signals".format(sum(redundant_signals) / car_df.shape[0]))

car_df = car_df[~redundant_signals]

0.009% redundant signals


In [18]:
check_signal_frequency(car_df)

accelerometer in 100.000% time had 1 signal per second (1 Hz)
gps_speed in 100.000% time had 1 signal per second (1 Hz)
battery_volt in 100.000% time had 1 signal per second (1 Hz)
coolant_temp in 100.000% time had 1 signal per second (1 Hz)
diagnostic_trouble_code in 100.000% time had 1 signal per second (1 Hz)
engine_load in 100.000% time had 1 signal per second (1 Hz)
intake_air_temp in 100.000% time had 1 signal per second (1 Hz)
imap in 100.000% time had 1 signal per second (1 Hz)
kpl in 100.000% time had 1 signal per second (1 Hz)
air_mass_rate in 100.000% time had 1 signal per second (1 Hz)
revolutions_per_min in 100.000% time had 1 signal per second (1 Hz)
speed in 100.000% time had 1 signal per second (1 Hz)
timing_advance in 100.000% time had 1 signal per second (1 Hz)
throttle_position in 100.000% time had 1 signal per second (1 Hz)


### Remove invalid accelerometer code
Accelerometer code should have 162 characters.  

In [19]:
car_df['accelerometer'].str.len().value_counts()

162    1953951
1       318498
Name: accelerometer, dtype: int64

In [20]:
car_df = car_df[car_df['accelerometer'].str.len() == 162]
car_df.shape

(1953951, 18)

### Decode Accelerometer Hex code

In [21]:
from decoding_acc import decode_acc_hex

decode_acc_hex_df = decode_acc_hex(car_df['accelerometer'], car_df['speed'])
car_df.loc[:, 'acc_x'] = decode_acc_hex_df['acc_x'].values
car_df.loc[:, 'acc_y'] = decode_acc_hex_df['acc_y'].values
car_df.loc[:, 'acc_z'] = decode_acc_hex_df['acc_z'].values

Decoding each accelerometer hex code.


100%|██████████████████████████████████████████████████████████████████████████| 1953951/1953951 [02:51<00:00, 11410.92it/s]
Aggregating 25Hz to 1Hz for accelerometer signals.: 100%|██████████████████████| 1953951/1953951 [01:22<00:00, 23779.72it/s]


In [22]:
car_df[car_df['acc_x'].isnull()]

,trip_id,device_id,timestamp,accelerometer,gps_speed,battery_volt,coolant_temp,diagnostic_trouble_code,engine_load,intake_air_temp,...,kpl,air_mass_rate,revolutions_per_min,speed,timing_advance,throttle_position,night_driving,acc_x,acc_y,acc_z


### Negative Trip Ids
There are 27% signals associated with negative trip ids. However, the shortest trip had over 1K records. A simple workaround is to treat them as individual trips.

In [23]:
print("Percentage of signals associated with negative trip_ids: {:,.3f}%".format(
    car_df[car_df['trip_id'] < 0].shape[0] / car_df.shape[0] * 100))

Percentage of signals associated with negative trip_ids: 27.016%


In [24]:
car_df[car_df['trip_id'] < 0][['trip_id', 'timestamp']]\
    .groupby('trip_id')\
    .agg({'timestamp': [len, min, max]})\
    .sort_values(('timestamp', 'len'))\
    .reset_index()

trip_id timestamp                                        
                  len                 min                 max
0       -74      1337 2017-11-18 22:50:21 2018-01-19 19:49:11
1       -45      1432 2017-11-20 17:12:53 2018-01-30 16:59:43
2        -2      1457 2017-11-30 06:29:41 2018-01-18 13:56:04
3       -39      1488 2017-11-20 20:09:41 2018-01-15 17:14:15
4       -20      1507 2017-11-21 18:47:19 2018-01-16 20:11:25
..      ...       ...                 ...                 ...
123      -9     10423 2017-11-25 09:15:14 2018-01-17 18:15:24
124     -65     10826 2017-11-19 14:07:53 2018-01-21 19:17:58
125     -88     10847 2017-11-18 20:59:00 2018-01-18 21:04:22
126    -105     11822 2017-11-18 19:59:53 2018-01-31 13:56:27
127     -15     15984 2017-11-22 07:15:11 2018-01-17 12:21:02

[128 rows x 4 columns]

### Signal Completeness
Only 3.48% trips have a complete signal history. Filling missing records is computationally expensive.

**A simply workaround is to treat timestamps as conseuctive signals.**

In [25]:
def unique_timestamps(x): 
    return len(set(x))

trip_df = car_df[['device_id', 'trip_id', 'timestamp']]\
    .groupby(['device_id', 'trip_id'])\
    .agg({'timestamp': [unique_timestamps, min, max]})\
    .sort_values(('timestamp', 'unique_timestamps'))\
    .reset_index()

In [26]:
trip_df['time_steps'] = (trip_df[('timestamp', 'max')] - trip_df[('timestamp', 'min')]).dt.seconds
trip_df.head(5)

device_id trip_id         timestamp                                          \
                    unique_timestamps                 min                 max   
0         8       1               150 2018-01-12 11:59:52 2018-01-12 12:02:26   
1        14       1               160 2017-12-21 10:59:18 2017-12-21 11:02:01   
2        10      62               161 2017-12-07 20:49:54 2017-12-07 20:52:46   
3        14       3               162 2017-12-24 09:37:22 2017-12-24 09:40:04   
4         3     100               162 2018-01-24 22:07:58 2018-01-24 22:10:42   

  time_steps  
              
0        154  
1        163  
2        172  
3        162  
4        164

In [27]:
trip_df['is_complete'] = (trip_df['time_steps'] == trip_df[('timestamp', 'unique_timestamps')])
print("{:,.3}% of trips have a complete signal history.".format(
    trip_df[(trip_df['is_complete'] == True)].shape[0] / trip_df.shape[0] * 100))

3.48% of trips have a complete signal history.


### Diagnostic Trouble Code

In [28]:
car_df['diagnostic_trouble_code'].value_counts()

 0      1953894
-127         55
 67           2
Name: diagnostic_trouble_code, dtype: int64

### Save out clean data

In [ ]:
car_df.to_csv('clean_ca')

#### Behavior Analysis
- trips completed per user
- total drive time per user
- drive time per trip
- trip distance
- trip history?


- day driving time vs. night driving time (night time can be 11pm to 4am)
- day driving is safter than night driving


- hardbreaks
- rash acceleration (hard accelerate)
- drive speed (km/hr), overspeeding
- idling with engine on (idling on a public road is dangerous. idling is not environmental friendly and burns the same amount of oil as driving.)

#### Customer Segmentation
Build driving behavior features for clustering, and analyse the commonality in each group. But customer profiling with more meaningful background information can make features more representative.

Beside, we don't have any customer information in the data.

#### Customer Safety Analysis
Some customers are more risky while driving. It causes a high chance to car accidents. It impacts car availability.

#### Car Usage Analysis
- what OBD data can tell?
- 